In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


In [2]:
cols_to_load = ['id1', 'id2', 'id3', 'id4', 'y', 'id5']  # adjust as needed
train_df = pd.read_parquet("Datasets/train_data.parquet")

offers_df = pd.read_parquet("Datasets/offer_metadata.parquet")










In [3]:
# Import function from offers_features.ipynb
%run offers_features.ipynb

# Generate and merge offer features
train_offer_df = generate_offer_features(train_df, offers_df)

[✔️] Offers merged and features engineered


In [38]:
train_offer_df.head()


,interaction_id,customer_id,offer_id,impression_timestamp,click,impression_date,offer_type,offer_code_type,offer_discount_value,offer_flag,offer_group,offer_metadata_flag,offer_extra_flag,f374,id8,offer_start,offer_end,offer_duration_days,discount_per_day
0,1366776_189706075_16-23_2023-11-02 22:22:00.042,1366776,189706075,2023-11-02 22:22:00.042,0,2023-11-02,n,2.0,2.0,None,1,None,g2baascbuyeectmpoanb1lo$bitscst,"Radio, Television, And Consumer Electronics St...",57310000,2023-11-01,2023-11-30 23:59:59,29.0,0.068966
1,1366776_89227_16-23_2023-11-01 23:51:24.999,1366776,89227,2023-11-01 23:51:24.999,0,2023-11-01,WE-WA&G,2.0,NaN,None,1,None,ga$scbuyeectsamo$ioompoawb4lo1sc(o$st,Liquor Stores,59210000,2023-11-01,2024-04-30 23:59:59,181.0,NaN
2,1366776_35046_16-23_2023-11-01 00:30:59.797,1366776,35046,2023-11-01 00:30:59.797,0,2023-11-01,t,2.0,10.0,None,1,None,g1baascbuyeectmpoatb1lo$bitscst,Beauty Shops,72310000,2023-11-01,2023-11-30 23:59:59,29.0,0.344828
3,1366776_6275451_16-23_2023-11-02 22:21:32.261,1366776,6275451,2023-11-02 22:21:32.261,0,2023-11-02,d,2.0,10.0,None,1,None,g1baascbuyeectmpoadb1lo$bitscst,Family Clothing Stores,56510500,2023-11-01,2023-11-30 23:59:59,29.0,0.344828
4,1366776_78053_16-23_2023-11-02 22:21:34.799,1366776,78053,2023-11-02 22:21:34.799,0,2023-11-02,e,2.0,8.0,None,1,None,g8baascbuyeectmpoaeb1lo$bitscst,"Miscellaneous Retail Stores, Nec",59991300,2023-11-01,2023-11-30 23:59:59,29.0,0.275862


In [4]:
# Load events
events_df = pd.read_parquet("Datasets/add_event.parquet")

# Import and apply event feature function
%run events_features.ipynb
train_offer_events_df = generate_event_features(train_offer_df, events_df)


[✓] Preprocessed events.
[✓] Merged offer-level event features into train_df.


,interaction_id,customer_id,offer_id,impression_timestamp,impression_date,click,f1,f2,f3,f4,...,discount_per_day,offer_event_count,unique_event_types,avg_event_hour,earliest_event,latest_event,event_span_days,Mobile_Timeline,OffersTab,Tiles
0,1366776_189706075_16-23_2023-11-02 22:22:00.042,1366776,189706075,2023-11-02 22:22:00.042,2023-11-02,0,1.0,None,None,None,...,0.068966,18238,3,11.653361,2023-11-01 00:02:48.157,2023-11-03 23:59:38.752,2,1338,10667,6233
1,1366776_89227_16-23_2023-11-01 23:51:24.999,1366776,89227,2023-11-01 23:51:24.999,2023-11-01,0,1.0,None,None,None,...,NaN,20780,3,11.495765,2023-11-01 00:02:07.254,2023-11-03 23:59:52.085,2,1399,12587,6794
2,1366776_35046_16-23_2023-11-01 00:30:59.797,1366776,35046,2023-11-01 00:30:59.797,2023-11-01,0,1.0,None,None,None,...,0.344828,18296,3,11.625656,2023-11-01 00:02:48.023,2023-11-03 23:59:03.252,2,1355,10717,6224
3,1366776_6275451_16-23_2023-11-02 22:21:32.261,1366776,6275451,2023-11-02 22:21:32.261,2023-11-02,0,1.0,None,None,None,...,0.344828,18012,3,11.683378,2023-11-01 00:02:48.743,2023-11-03 23:58:58.116,2,1401,10340,6271
4,1366776_78053_16-23_2023-11-02 22:21:34.799,1366776,78053,2023-11-02 22:21:34.799,2023-11-02,0,1.0,None,None,None,...,0.275862,18428,3,11.625950,2023-11-01 00:02:49.701,2023-11-03 23:59:05.369,2,1397,10645,6386
5,1366776_31794_16-23_2023-11-02 22:21:32.333,1366776,31794,2023-11-02 22:21:32.333,2023-11-02,0,1.0,None,None,None,...,0.206897,18458,3,11.602557,2023-11-01 00:02:47.456,2023-11-03 23:59:25.985,2,1336,10694,6428
6,1366776_664919_16-23_2023-11-01 23:51:25.022,1366776,664919,2023-11-01 23:51:25.022,2023-11-01,0,1.0,None,None,None,...,NaN,55057,3,12.233104,2023-10-22 00:00:08.891,2023-11-03 23:58:56.551,12,5367,29981,19709
7,1366776_481197_16-23_2023-11-02 22:21:46.374,1366776,481197,2023-11-02 22:21:46.374,2023-11-02,0,1.0,None,None,None,...,0.172414,18226,3,11.598102,2023-11-01 00:02:47.878,2023-11-03 23:59:23.927,2,1311,10621,6294
8,1366776_78680_16-23_2023-11-02 22:21:03.978,1366776,78680,2023-11-02 22:21:03.978,2023-11-02,0,1.0,None,None,None,...,NaN,58751,3,12.166670,2023-10-22 00:00:48.471,2023-11-03 23:59:18.682,12,5701,32138,20912
9,1366776_960643_16-23_2023-11-02 22:22:55.276,1366776,960643,2023-11-02 22:22:55.276,2023-11-02,0,1.0,None,None,None,...,NaN,26133,3,11.923201,2023-10-30 00:02:58.072,2023-11-03 23:58:36.823,4,2022,14781,9330


In [5]:
transactions_df = pd.read_parquet("Datasets/add_trans.parquet")
transactions_df.head(20)

,id2,f367,f368,f369,f370,f371,f372,id8,f374
0,2896709,15.60,PBR,D,2023-10-16,19:16:52,202310,59639998,DSE
1,2855047,6.40,PR,D,2023-10-14,13:01:16,202310,59639998,DSE
2,2497175,13.99,PBR,D,2023-10-14,00:31:48,202310,59639998,DSE
3,2655364,15.14,PGC,D,2023-10-13,12:37:25,202310,59639998,DSE
4,2855047,2.12,PR,D,2023-10-09,16:51:21,202310,59639998,DSE
5,2390106,15.42,PR,D,2023-10-22,17:24:37,202310,59639998,DSE
6,2166784,13.99,PR,D,2023-10-06,16:27:55,202310,59639998,DSE
7,2487698,2.99,PR,D,2023-10-15,20:45:06,202310,59639998,DSE
8,2385402,50.00,PR,D,2023-10-02,13:10:06,202310,59639998,DSE
9,2385402,99.99,PR,D,2023-10-27,17:31:55,202310,59639998,DSE


In [6]:
# Run the transactions feature notebook
%run transaction_features.ipynb

# Apply global transaction features to train data
train_OET_df = generate_global_transaction_features(train_offer_events_df, transactions_df)


[✓] Global transaction stats merged into train set.


,interaction_id,customer_id,offer_id,impression_timestamp,impression_date,click,f1,f2,f3,f4,...,latest_event,event_span_days,Mobile_Timeline,OffersTab,Tiles,global_avg_transaction,global_max_transaction,global_min_transaction,global_top_category,global_top_type
0,1366776_189706075_16-23_2023-11-02 22:22:00.042,1366776,189706075,2023-11-02 22:22:00.042,2023-11-02,0,1.0,None,None,None,...,2023-11-03 23:59:38.752,2,1338,10667,6233,182.232674,1116928.8,0.01,PR,D
1,1366776_89227_16-23_2023-11-01 23:51:24.999,1366776,89227,2023-11-01 23:51:24.999,2023-11-01,0,1.0,None,None,None,...,2023-11-03 23:59:52.085,2,1399,12587,6794,182.232674,1116928.8,0.01,PR,D
2,1366776_35046_16-23_2023-11-01 00:30:59.797,1366776,35046,2023-11-01 00:30:59.797,2023-11-01,0,1.0,None,None,None,...,2023-11-03 23:59:03.252,2,1355,10717,6224,182.232674,1116928.8,0.01,PR,D
3,1366776_6275451_16-23_2023-11-02 22:21:32.261,1366776,6275451,2023-11-02 22:21:32.261,2023-11-02,0,1.0,None,None,None,...,2023-11-03 23:58:58.116,2,1401,10340,6271,182.232674,1116928.8,0.01,PR,D
4,1366776_78053_16-23_2023-11-02 22:21:34.799,1366776,78053,2023-11-02 22:21:34.799,2023-11-02,0,1.0,None,None,None,...,2023-11-03 23:59:05.369,2,1397,10645,6386,182.232674,1116928.8,0.01,PR,D


In [60]:
train_OET_df.shape

(770164, 399)

In [7]:
train_OET_df.isnull().sum().sort_values(ascending=False).head(200)


f136                   770164
offer_metadata_flag    770164
offer_flag             770164
f135                   770164
f122                   770164
                        ...  
f162                    40714
f163                    40714
f164                    40714
f165                    40714
f171                    40714
Length: 200, dtype: int64

In [8]:
# Drop columns with more than 90% null values
null_thresh = 0.50
null_fraction = train_OET_df.isnull().mean()
cols_to_drop = null_fraction[null_fraction > null_thresh].index.tolist()

train_OET_df.drop(columns=cols_to_drop, inplace=True)
print(f"Dropped {len(cols_to_drop)} columns with >60% missing values")


Dropped 74 columns with >60% missing values


In [9]:
# For numeric columns: fill with median
for col in train_OET_df.select_dtypes(include='number').columns:
    train_OET_df[col].fillna(train_OET_df[col].median(), inplace=True)

# For categorical columns: fill with a new category like 'unknown'
for col in train_OET_df.select_dtypes(include='object').columns:
    train_OET_df[col].fillna('unknown', inplace=True)


C:\Users\astit\AppData\Local\Temp\ipykernel_23648\3284056172.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_OET_df[col].fillna(train_OET_df[col].median(), inplace=True)
C:\Users\astit\AppData\Local\Temp\ipykernel_23648\3284056172.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves a

In [70]:
train_OET_df.head(20)

,interaction_id,customer_id,offer_id,impression_timestamp,impression_date,click,f1,f2,f5,f6,...,latest_event,event_span_days,Mobile_Timeline,OffersTab,Tiles,global_avg_transaction,global_max_transaction,global_min_transaction,global_top_category,global_top_type
0,1366776_189706075_16-23_2023-11-02 22:22:00.042,1366776,189706075,2023-11-02 22:22:00.042,2023-11-02,0,1.0,unknown,unknown,unknown,...,2023-11-03 23:59:38.752,2,1338,10667,6233,182.232674,1116928.8,0.01,PR,D
1,1366776_89227_16-23_2023-11-01 23:51:24.999,1366776,89227,2023-11-01 23:51:24.999,2023-11-01,0,1.0,unknown,unknown,unknown,...,2023-11-03 23:59:52.085,2,1399,12587,6794,182.232674,1116928.8,0.01,PR,D
2,1366776_35046_16-23_2023-11-01 00:30:59.797,1366776,35046,2023-11-01 00:30:59.797,2023-11-01,0,1.0,unknown,unknown,unknown,...,2023-11-03 23:59:03.252,2,1355,10717,6224,182.232674,1116928.8,0.01,PR,D
3,1366776_6275451_16-23_2023-11-02 22:21:32.261,1366776,6275451,2023-11-02 22:21:32.261,2023-11-02,0,1.0,unknown,unknown,unknown,...,2023-11-03 23:58:58.116,2,1401,10340,6271,182.232674,1116928.8,0.01,PR,D
4,1366776_78053_16-23_2023-11-02 22:21:34.799,1366776,78053,2023-11-02 22:21:34.799,2023-11-02,0,1.0,unknown,unknown,unknown,...,2023-11-03 23:59:05.369,2,1397,10645,6386,182.232674,1116928.8,0.01,PR,D
5,1366776_31794_16-23_2023-11-02 22:21:32.333,1366776,31794,2023-11-02 22:21:32.333,2023-11-02,0,1.0,unknown,unknown,unknown,...,2023-11-03 23:59:25.985,2,1336,10694,6428,182.232674,1116928.8,0.01,PR,D
6,1366776_664919_16-23_2023-11-01 23:51:25.022,1366776,664919,2023-11-01 23:51:25.022,2023-11-01,0,1.0,unknown,unknown,unknown,...,2023-11-03 23:58:56.551,12,5367,29981,19709,182.232674,1116928.8,0.01,PR,D
7,1366776_481197_16-23_2023-11-02 22:21:46.374,1366776,481197,2023-11-02 22:21:46.374,2023-11-02,0,1.0,unknown,unknown,unknown,...,2023-11-03 23:59:23.927,2,1311,10621,6294,182.232674,1116928.8,0.01,PR,D
8,1366776_78680_16-23_2023-11-02 22:21:03.978,1366776,78680,2023-11-02 22:21:03.978,2023-11-02,0,1.0,unknown,unknown,unknown,...,2023-11-03 23:59:18.682,12,5701,32138,20912,182.232674,1116928.8,0.01,PR,D
9,1366776_960643_16-23_2023-11-02 22:22:55.276,1366776,960643,2023-11-02 22:22:55.276,2023-11-02,0,1.0,unknown,unknown,unknown,...,2023-11-03 23:58:36.823,4,2022,14781,9330,182.232674,1116928.8,0.01,PR,D


In [10]:
print(train_OET_df['click'].dtype)
# Convert string '0'/'1' to integer 0/1
train_OET_df['click'] = train_OET_df['click'].astype(int)

print("Count of y = 1:", (train_OET_df['click'] == 1).sum())
print("Count of y = 0:", (train_OET_df['click'] == 0).sum())


object
Count of y = 1: 37051
Count of y = 0: 733113


In [11]:
train_OET_df.to_parquet("Outputs/final_train_df.parquet", index=False)


FEATURE ENGINEERING AND PREPROCESSING DONE UPTILL HERE 